# Testing the Algorithm 
To test if the algorithm works properly and in alignment with the instructions from the Project Guidelines, **3 tests** will be run. These include:  
- **2 functional tests**: does the algorithm output correct solutions if adequate inputs are provided? This especially pertains to the amount of transactions (are they really minimized) and the path that it outputs (are the correct amounts transferred to and from the correct people). Functional tests will be verified through comparison with the actual Splitwise-app. 
- **1 performance test**: does the algorithm scale well? I.e. can it solve transactions with 10, 25 or 50 different involved people in under a second?

## Exception Tests


## Functional Tests

## Performance Test
Does the algorithm solve transactions with more than 15 people in less than 1 second? 

In [1]:
import random, time
from splitwise import OptimalSplit

def generate_transactions(num_people, num_transactions_per_person=10, max_amount=100):
    """
    Generate a list of transactions of the form [lender, borrower, amount].

    Args:
      - num_people: how many unique people (named "P0", "P1", …).
      - num_transactions_per_person: average number of transactions per person.
      - max_amount: upper bound for a single transaction’s amount.
    """
    people = [f"P{i}" for i in range(num_people)]
    transactions = []
    for _ in range(num_people * num_transactions_per_person):
        lender, borrower = random.sample(people, 2)
        amount = round(random.uniform(1, max_amount), 2)
        transactions.append([lender, borrower, amount])
    return transactions

transactions_1 = generate_transactions(5, 5, 100)
transactions_2 = generate_transactions(15, 5, 100)
transactions_3 = generate_transactions(50, 5, 100)

print(transactions_2)


[['P9', 'P12', 9.25], ['P6', 'P7', 19.57], ['P6', 'P3', 15.47], ['P0', 'P1', 40.2], ['P13', 'P6', 24.48], ['P6', 'P7', 40.31], ['P10', 'P7', 64.78], ['P12', 'P8', 20.62], ['P1', 'P4', 90.2], ['P10', 'P14', 91.39], ['P1', 'P10', 46.65], ['P6', 'P10', 18.49], ['P5', 'P0', 41.0], ['P0', 'P2', 75.51], ['P7', 'P10', 47.96], ['P0', 'P10', 79.3], ['P13', 'P10', 50.13], ['P0', 'P12', 65.41], ['P13', 'P5', 66.99], ['P8', 'P10', 93.53], ['P7', 'P14', 26.7], ['P11', 'P6', 51.94], ['P14', 'P13', 55.52], ['P5', 'P7', 86.02], ['P9', 'P6', 90.04], ['P10', 'P11', 14.92], ['P2', 'P13', 50.56], ['P8', 'P7', 40.41], ['P1', 'P11', 87.09], ['P0', 'P12', 96.83], ['P5', 'P2', 14.38], ['P7', 'P0', 17.52], ['P11', 'P1', 17.71], ['P9', 'P1', 58.4], ['P7', 'P2', 81.17], ['P1', 'P11', 74.58], ['P4', 'P13', 15.6], ['P2', 'P10', 33.29], ['P13', 'P10', 1.04], ['P14', 'P10', 39.04], ['P7', 'P4', 69.16], ['P7', 'P8', 53.09], ['P14', 'P2', 16.59], ['P6', 'P4', 92.24], ['P13', 'P14', 71.17], ['P11', 'P3', 74.55], ['P2',

In [2]:
solver = OptimalSplit()
for n in (5, 15, 50):
    txns = generate_transactions(n)
    start = time.perf_counter()
    solver.minTransfers(txns)
    elapsed = time.perf_counter() - start
    print(f"{n:2d} people → {elapsed:.4f}s")
    assert elapsed < 1.0, f"Took too long for {n} people"

defaultdict(<class 'int'>, {'P1': 148.11999999999995, 'P2': -25.85000000000001, 'P4': -23.520000000000003, 'P3': -153.55000000000004, 'P0': 54.80000000000005})
{'P1': 148.11999999999995, 'P2': -25.85000000000001, 'P4': -23.520000000000003, 'P3': -153.55000000000004, 'P0': 54.80000000000005}
 5 people → 0.0001s
defaultdict(<class 'int'>, {'P0': -7.6699999999999955, 'P1': -5.65999999999997, 'P8': 87.62999999999995, 'P14': -160.92999999999995, 'P4': 74.96, 'P13': -186.24, 'P9': 108.80999999999997, 'P7': -293.02000000000004, 'P6': 311.0899999999999, 'P12': 308.36, 'P2': -109.28, 'P5': 334.07000000000005, 'P10': -220.8, 'P3': -9.449999999999994, 'P11': -231.87})
{'P0': -7.6699999999999955, 'P1': -5.65999999999997, 'P8': 87.62999999999995, 'P14': -160.92999999999995, 'P4': 74.96, 'P13': -186.24, 'P9': 108.80999999999997, 'P7': -293.02000000000004, 'P6': 311.0899999999999, 'P12': 308.36, 'P2': -109.28, 'P5': 334.07000000000005, 'P10': -220.8, 'P3': -9.449999999999994, 'P11': -231.87}
15 peopl

In [3]:
transactions = [['P0', 'P8', 57.84], ['P5', 'P3', 18.98], ['P4', 'P14', 82.19], ['P9', 'P2', 37.96], ['P4', 'P11', 90.33], ['P11', 'P9', 74.92], ['P5', 'P6', 79.66], ['P11', 'P14', 2.82], ['P7', 'P8', 35.58], ['P11', 'P10', 61.73], ['P7', 'P1', 32.3], ['P3', 'P10', 92.2], ['P9', 'P4', 20.2], ['P5', 'P8', 11.16], ['P11', 'P10', 78.52], ['P11', 'P6', 67.42], ['P12', 'P7', 60.01], ['P0', 'P6', 48.12], ['P5', 'P4', 35.8], ['P10', 'P5', 12.62], ['P2', 'P0', 28.55], ['P11', 'P1', 16.15], ['P6', 'P4', 10.49], ['P13', 'P4', 21.64], ['P9', 'P13', 38.32], ['P4', 'P8', 19.9], ['P7', 'P2', 91.32], ['P8', 'P6', 77.25], ['P2', 'P11', 72.95], ['P7', 'P10', 29.73], ['P13', 'P4', 6.42], ['P7', 'P10', 42.13], ['P12', 'P4', 53.18], ['P11', 'P9', 2.27], ['P6', 'P11', 92.43], ['P1', 'P4', 20.77], ['P11', 'P12', 73.13], ['P5', 'P7', 77.82], ['P5', 'P11', 63.83], ['P13', 'P10', 73.35], ['P3', 'P10', 63.91], ['P11', 'P13', 90.25], ['P13', 'P8', 39.55], ['P5', 'P3', 59.41], ['P0', 'P12', 5.77], ['P9', 'P13', 2.21], ['P9', 'P6', 48.88], ['P12', 'P5', 31.7], ['P9', 'P0', 39.94], ['P5', 'P12', 22.19], ['P2', 'P6', 83.47], ['P1', 'P13', 37.59], ['P0', 'P5', 32.21], ['P11', 'P8', 31.23], ['P3', 'P13', 18.99], ['P13', 'P0', 3.99], ['P5', 'P9', 93.63], ['P4', 'P6', 36.84], ['P0', 'P11', 87.0], ['P8', 'P1', 10.07], ['P13', 'P7', 6.87], ['P14', 'P13', 81.91], ['P5', 'P2', 71.36], ['P13', 'P10', 26.41], ['P12', 'P14', 46.7], ['P8', 'P6', 87.2], ['P5', 'P7', 76.77], ['P10', 'P13', 69.5], ['P6', 'P1', 41.12], ['P10', 'P0', 40.94], ['P4', 'P3', 50.6], ['P6', 'P14', 34.04], ['P0', 'P12', 94.45], ['P8', 'P11', 80.79], ['P4', 'P8', 56.86]]

solver.minTransfers(transactions)

defaultdict(<class 'int'>, {'P0': 211.97000000000003, 'P8': 3.190000000000012, 'P5': 534.08, 'P3': 46.10999999999999, 'P4': 168.21999999999997, 'P14': -83.84, 'P9': 16.689999999999998, 'P2': -15.669999999999987, 'P11': 11.109999999999943, 'P6': -350.76, 'P7': 9.58999999999999, 'P10': -344.91999999999996, 'P1': -41.27999999999999, 'P12': -3.9499999999999886, 'P13': -160.54000000000002})
{'P0': 211.97000000000003, 'P8': 3.190000000000012, 'P5': 534.08, 'P3': 46.10999999999999, 'P4': 168.21999999999997, 'P14': -83.84, 'P9': 16.689999999999998, 'P2': -15.669999999999987, 'P11': 11.109999999999943, 'P6': -350.76, 'P7': 9.58999999999999, 'P10': -344.91999999999996, 'P1': -41.27999999999999, 'P12': -3.9499999999999886, 'P13': -160.54000000000002}


[['P5', 'P6', 350.76],
 ['P0', 'P10', 211.97000000000003],
 ['P5', 'P13', 160.54000000000002],
 ['P4', 'P10', 132.94999999999993],
 ['P3', 'P14', 46.10999999999999],
 ['P4', 'P1', 35.27000000000004],
 ['P5', 'P14', 22.78000000000003],
 ['P9', 'P2', 15.669999999999987],
 ['P11', 'P14', 11.109999999999943],
 ['P7', 'P1', 6.009999999999948],
 ['P7', 'P12', 3.580000000000041],
 ['P8', 'P14', 3.190000000000012],
 ['P9', 'P14', 0.650000000000027],
 ['P9', 'P12', 0.3699999999999477]]

In [4]:
result = solver.minTransfers(transactions_3)
result

defaultdict(<class 'int'>, {'P40': 1.9899999999999807, 'P42': 257.46000000000004, 'P23': -331.44, 'P43': -38.23000000000002, 'P32': -134.96000000000004, 'P29': -159.95000000000002, 'P27': -78.36000000000001, 'P37': -50.13999999999999, 'P1': 252.51, 'P48': -191.32999999999998, 'P49': 228.45999999999995, 'P0': -190.16, 'P7': -103.77000000000005, 'P3': -216.33999999999997, 'P38': -105.04, 'P34': -218.13, 'P13': -168.01000000000002, 'P39': 441.93, 'P30': -287.1, 'P19': -253.26999999999998, 'P15': -71.97999999999998, 'P4': 362.2, 'P22': -118.2, 'P11': 79.97999999999999, 'P9': -234.26999999999998, 'P24': -197.57, 'P44': 287.48, 'P35': 190.85, 'P6': 470.73, 'P28': -398.9, 'P20': -69.79000000000002, 'P26': -36.84, 'P33': 40.24999999999999, 'P25': 94.61000000000001, 'P14': 187.67000000000002, 'P17': -452.39000000000004, 'P46': -69.00999999999999, 'P2': 51.86000000000001, 'P16': -87.36999999999996, 'P36': -140.53, 'P5': 356.84, 'P8': 64.69, 'P45': 349.77000000000004, 'P18': 78.99999999999997, 'P

[['P6', 'P17', 452.39000000000004],
 ['P39', 'P28', 398.9],
 ['P4', 'P23', 331.44],
 ['P5', 'P30', 287.1],
 ['P21', 'P19', 253.26999999999998],
 ['P45', 'P41', 241.58],
 ['P12', 'P9', 234.26999999999998],
 ['P44', 'P34', 218.13],
 ['P42', 'P3', 216.33999999999997],
 ['P1', 'P24', 197.57],
 ['P49', 'P48', 191.32999999999998],
 ['P35', 'P0', 190.16],
 ['P14', 'P13', 168.01000000000002],
 ['P10', 'P29', 159.95000000000002],
 ['P45', 'P36', 108.19000000000003],
 ['P47', 'P32', 101.97],
 ['P21', 'P22', 97.47000000000003],
 ['P25', 'P38', 94.61000000000001],
 ['P11', 'P7', 79.97999999999999],
 ['P18', 'P16', 78.99999999999997],
 ['P5', 'P27', 69.73999999999995],
 ['P44', 'P15', 69.35000000000002],
 ['P8', 'P20', 64.69],
 ['P1', 'P46', 54.94],
 ['P12', 'P31', 54.77999999999997],
 ['P2', 'P37', 50.13999999999999],
 ['P39', 'P43', 38.23000000000002],
 ['P42', 'P26', 36.84],
 ['P33', 'P32', 32.99000000000004],
 ['P49', 'P36', 32.339999999999975],
 ['P4', 'P7', 23.790000000000063],
 ['P14', 'P22'